## Setup

### Config

In [ ]:
USE_LOCAL_AALPY = True

GROUP_BY_MODEL = False

### Imports

In [ ]:
import sys
sys.path.append(f"..")
sys.path.append(r"../pmsat-inference")
if USE_LOCAL_AALPY:
    sys.path = [r"../../AALpy"] + sys.path

from evaluation.utils import print_results_info, print_results_info_per_alg, TracedMooreSUL
from evaluation.charts import *
from IPython.display import display, Markdown, Latex

## Loading Results

### Loading

In [ ]:


results_dir = r"../learning_results_11"  #  python evaluation/learn_automata.py -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(NO_REP)" "APMSL(NO_REP, RW)" -o "None" -n 3 -ns 3-8 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state"
results_dir = r"../learning_results_12"  #  python evaluation/learn_automata.py -a "APMSL" "APMSL(ONLY_RW)" -o "None" -n 3 -ns 3-6 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 1.0 --glitch_mode "enter_random_state"  # mit 'sul_transitions'
results_dir = r"../learning_results_16"  # python evaluation/learn_automata.py -o "None" -n 3 -ns 3-6 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 1.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(TC)" "APMSL(NO_REP, RW)"
results_dir = r"../learning_results_17"  # python evaluation/learn_automata.py -o "None" "Random" -n 3 -ns 3-8 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(TC)" "APMSL(NO_REP, RW)"
results_dir = r"../learning_results_18"  # python evaluation/learn_automata.py -o "None" -n 3 -ns 4-6 -ni 3 -no 3 --learn_num_times 2 --glitch_percent 0.0 0.1 --glitch_mode "enter_random_state" -a "APMSL(tc=0)" "APMSL(tc=100)" "APMSL(tc=500)"
results_dir = r"../learning_results_19"  # python evaluation/learn_automata.py -o "None" -n 3 -ns 4-6 -ni 3 -no 3 --learn_num_times 2 --glitch_percent 0.0 1.0 --glitch_mode "enter_random_state" -a "APMSL(tc=0)" "APMSL(tc=100)" "APMSL(tc=500)" "APMSL(tc=1000)" "APMSL(ONLY_RW)"
results_dir = r"../learning_results_20"  # no deduplicate_traces!  # python evaluation/learn_automata.py -o "None" -n 3 -ns 3-6 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 1.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(tc=500)" "APMSL(NO_REP, RW)"
# 22 # python evaluation/learn_automata.py -o "None" -n 3 -ns 7 -ni 3 -no 3 --learn_num_times 2 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(tc=500)" "APMSL(NO_REP, RW)"  # (also no duplication)
# 23 # python evaluation/learn_automata.py -o "None" -n 3 -ns 7 -ni 3 -no 3 --learn_num_times 2 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(tc=500)" "APMSL(NO_REP, RW)"  # (also no duplication) (only replay glitches on peak)
# 24 # python evaluation/learn_automata.py -o "None" "Random" -n 3 -ns 3-6 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ONLY_RW)" "APMSL(RW)" "APMSL(tc=500)" "APMSL(NO_REP, RW)" # no deduplication, only replay glitches on peak
# 25 # python evaluation/learn_automata.py -o "None" -n 10 -ns 3-9 -ni 3 -no 3 --learn_num_times 3 --glitch_percent 0.0 1.0 5.0 --glitch_mode "enter_random_state" -a "APMSL" "APMSL(ddt=False)" "APMSL(ddt=False, orpg=True)" "APMSL(ONLY_RW)" "APMSL(ONLY_RW, ddt=True)" "APMSL(RW)" "APMSL(RW, ddt=False)" "APMSL(RW, ddt=False, orgp=True)" "APMSL(tc=500)" "APMSL(tc=500, ddt=False)" "APMSL(NO_REP, RW, ddt=False)"  # interrupted when starting 7state 
# 26: python evaluation/learn_automata.py --learn_all_automata_from_dir example_automata/moore -a "APMSL(RW)" -o "Random" "None"  # terminated by MemoryError
# 34:  python evaluation/learn_automata.py --files example_automata/moore -a "APMSL(RW)" "APMSL(ONLY_RW)" "APMSL" -o "Random" "None" --glitch_mode "enter_random_state" --glitch_percent 0.0 1.0

results_dir = r"../learning_results_49"

results = load_results(results_dir)
print(f"Loaded {len(results)} results!")

### Cleaning

In [ ]:
def remove_result_if(result):
    if result["algorithm_name"] == "APMSL('RW', ddt=False, orgp=True)":
        # typo in orpg -> always failed. remove
        return True
        
    return False

def postprocess_result(result):
    if result["algorithm_name"] == "APMSL('ONLY_RW', ddt=True)":
        # ddt=True is default anyways, so we can combine
        result["algorithm_name"] = "APMSL('ONLY_RW')"
    
    if result["algorithm_name"].endswith("()"):
        # remove empty braces in alg name
        result["algorithm_name"] = result["algorithm_name"][:-2]

    if "'" in result["algorithm_name"]:
        # remove quotes in alg name
        result["algorithm_name"] = result["algorithm_name"].replace("'", "")

    return result

def filter_results(results):
    return [postprocess_result(r) for r in results if not remove_result_if(r)]
    

results = filter_results(results)

## Basic Summary

### Overall

In [ ]:
print_results_info(results)

### Per Algorithm

In [ ]:
print_results_info_per_alg(results, exclude_results_with_eq_oracle=True)

## Statistics

### Overall

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

#if GROUP_BY_MODEL:
#    kwargs["group_by"] = "original_automaton"

bar_chart_per_algorithm_and_oracle(results, "learned_correctly", stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, "Precision", stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, "Recall", stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, "F-Score", stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, "Precision (all traces)", stat_method=np.mean, only_if=is_valid_result, **kwargs)

### Per number of states

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"

min_num_states = min(r['original_automaton_size'] for r in results)
max_num_states = max(r['original_automaton_size'] for r in results)

def valid_and_correct_num_states(res, num_states):
    if not is_valid_result(res):
        return False
    return res["original_automaton_size"] == num_states
    

for num_states in range(min_num_states, max_num_states + 1):
    bar_chart_per_algorithm_and_oracle(results, "learned_correctly", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"learned_correctly for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "Precision", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns),
                                       title=f"Precision for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "Recall", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"Recall for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "F-Score", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"F-Score for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "Precision (all traces)", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"Precision (all traces) for {num_states} states", **kwargs)

### Line chart over states (one plot per algorithm)

In [ ]:
STATS = 'learned_correctly', 'Precision', 'Recall', 'F-Score', 'Precision (all traces)'
ALGORITHMS = 'all'
ORACLES = 'all'
CHART_TYPE = 'line'  # one of 'bar', 'line'

algs = ALGORITHMS
if ALGORITHMS == 'all':
    algs = get_algorithm_names(results)

eq_oracles = ORACLES    
if ORACLES == 'all':
    eq_oracles = get_oracle_names(results)

glitch_percentages = get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result) 

for glitch_percentage in glitch_percentages:
    display(Markdown(f'#### {glitch_percentage}% glitches'))
    for (a, o) in itertools.product(algs, eq_oracles):
        display(Markdown(f'##### {a}, {o}'))
        
        if CHART_TYPE == 'bar':
            for stat in STATS:
                bar_chart_per_number_of_original_states(results, stat, stat_method=np.mean, 
                                                        only_if=lambda res: res['algorithm_name'] == a 
                                                                            and res['oracle'] == o
                                                                            and res['glitch_percent'] == glitch_percentage
                                                                            and is_valid_result(res))
                
        elif CHART_TYPE == 'line':
            line_chart_per_number_of_original_states(results, *STATS, stat_method=np.mean,
                                                     only_if=lambda res: res['algorithm_name'] == a
                                                                         and res['oracle'] == o
                                                                         and res['glitch_percent'] == glitch_percentage
                                                                         and is_valid_result(res))
        else:
            raise NotImplementedError(f"Chart type {CHART_TYPE} not implemented")

### Line chart over states (one plot per stat)

In [ ]:
STATS = 'learned_correctly', 'Precision', 'Recall', 'F-Score', 'Precision (all traces)'
ALGORITHMS = 'all'
ORACLES = 'all'
CHART_TYPE = 'line'  # one of 'bar', 'line'

algs = ALGORITHMS
if ALGORITHMS == 'all':
    algs = get_algorithm_names(results)

eq_oracles = ORACLES    
if ORACLES == 'all':
    eq_oracles = get_oracle_names(results)

glitch_percentages = get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result) 

for glitch_percentage in glitch_percentages:
    display(Markdown(f'#### {glitch_percentage}% glitches'))
    for stat in STATS:
        display(Markdown(f'##### {stat}'))
        
        if CHART_TYPE == 'bar':
            for (a, o) in itertools.product(algs, eq_oracles):
                bar_chart_per_number_of_original_states(results, stat, stat_method=np.mean, 
                                                        only_if=lambda res: res['algorithm_name'] == a 
                                                                            and res['oracle'] == o
                                                                            and res['glitch_percent'] == glitch_percentage
                                                                            and is_valid_result(res))
                
        elif CHART_TYPE == 'line':
            line_chart_per_number_of_original_states_per_alg_and_orac(results, stat, stat_method=np.mean,
                                                                      algs=algs, oracles=eq_oracles,
                                                                      only_if=lambda res: is_valid_result(res)
                                                                                          and res['glitch_percent'] == glitch_percentage)
        else:
            raise NotImplementedError(f"Chart type {CHART_TYPE} not implemented")

## Number of Steps/Queries  

### Overall bar charts per algorithm

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"


bar_chart_per_algorithm_and_oracle(results, 'steps_learning', stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, 'queries_learning', stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, 'steps_eq_oracle', stat_method=np.mean, only_if=is_valid_result, **kwargs)
bar_chart_per_algorithm_and_oracle(results, 'queries_eq_oracle', stat_method=np.mean, only_if=is_valid_result, **kwargs)

### Overall bar charts per algorithm, without MAT

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"

def non_mat(res):
    if not is_valid_result(res):
        return False
    return res["oracle"] in (None, "None")

bar_chart_per_algorithm_and_oracle(results, 'steps_learning', stat_method=np.mean, only_if=non_mat, **kwargs)
bar_chart_per_algorithm_and_oracle(results, 'queries_learning', stat_method=np.mean, only_if=non_mat, **kwargs)

### Seperate bar per algorithm for each number of states)

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"


min_num_states = min(r['original_automaton_size'] for r in results)
max_num_states = max(r['original_automaton_size'] for r in results)

def valid_and_correct_num_states(res, num_states):
    if not is_valid_result(res):
        return False
    return res["original_automaton_size"] == num_states
    

for num_states in range(min_num_states, max_num_states + 1):
    bar_chart_per_algorithm_and_oracle(results, "steps_learning", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"steps_learning for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "queries_learning", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns),
                                       title=f"queries_learning for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "steps_eq_oracle", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"steps_eq_oracle for {num_states} states", **kwargs)
    bar_chart_per_algorithm_and_oracle(results, "queries_eq_oracle", stat_method=np.mean, 
                                       only_if=lambda res, ns=num_states: valid_and_correct_num_states(res, ns), 
                                       title=f"queries_eq_oracle for {num_states} states", **kwargs)

### Line charts over number of states

In [ ]:
METRICS = 'steps_learning', 'queries_learning'
ALGORITHMS = 'all'
ORACLES = 'all'
CHART_TYPE = 'line'  # one of 'bar', 'line'

algs = ALGORITHMS
if ALGORITHMS == 'all':
    algs = get_algorithm_names(results)

eq_oracles = ORACLES    
if ORACLES == 'all':
    eq_oracles = get_oracle_names(results)

glitch_percentages = get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result) 

for glitch_percentage in glitch_percentages:
    display(Markdown(f'#### {glitch_percentage}% glitches'))
    for metric in METRICS:
        display(Markdown(f'##### {metric}'))
        
        if CHART_TYPE == 'bar':
            for (a, o) in itertools.product(algs, eq_oracles):
                bar_chart_per_number_of_original_states(results, metric, stat_method=np.mean, 
                                                        only_if=lambda res: res['algorithm_name'] == a 
                                                                            and res['oracle'] == o
                                                                            and res['glitch_percent'] == glitch_percentage
                                                                            and is_valid_result(res))
                
        elif CHART_TYPE == 'line':
            line_chart_per_number_of_original_states_per_alg_and_orac(results, metric, stat_method=np.mean,
                                                                      algs=algs, oracles=eq_oracles,
                                                                      only_if=lambda res: is_valid_result(res)
                                                                                          and res['glitch_percent'] == glitch_percentage)
        else:
            raise NotImplementedError(f"Chart type {CHART_TYPE} not implemented")

## Number of additional traces

### Stacked bars

In [ ]:
glitch_percentages = get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result) 

for glitch_percentage in glitch_percentages:
    display(Markdown(f'#### {glitch_percentage}% glitches'))
    stacked_bar_chart_add_traces_per_algorithm_and_oracle(results, None, None, 
                                                          only_if=lambda res: is_valid_result(res) and res['glitch_percent'] == glitch_percentage)

### Bars

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"


for add_traces_name in ['preprocessing_input_completeness', 'postprocessing_glitch', 'postprocessing_window_cex', 'postprocessing_replay', 'postprocessing_random_walks', 'postprocessing_cex', 'postprocessing_transition_coverage']:
    def get(res, name=add_traces_name):
        summed_num_traces = 0
        for learning_round, info in res["detailed_learning_info"].items():
            summed_num_traces += len(info.get(f"additional_traces_{name}", []))
        return summed_num_traces
    get.__name__ = f'len({add_traces_name})'
    bar_chart_per_algorithm_and_oracle(results, get, stat_method=np.mean, only_if=is_valid_result, **kwargs)
    bar_chart_per_algorithm_and_oracle(results, get, stat_method=np.sum, only_if=is_valid_result, **kwargs)

### Line charts over learning rounds

In [ ]:
import functools    
def num_traces(result: dict, add_traces_name: str):
    return len(result.get(add_traces_name, []))
    
for add_traces in ('preprocessing_input_completeness', 'postprocessing_glitch', 'postprocessing_replay', 'postprocessing_random_walks', 'postprocessing_cex'):
    func = functools.partial(num_traces, add_traces_name=f"additional_traces_{add_traces}")
    func.__name__ = f"len({add_traces})"
    line_chart_over_learning_rounds(results, key=func, only_if=is_valid_result)

## Time-Outs

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"


bar_chart_per_algorithm_and_oracle(results, "timed_out", stat_method=np.sum, only_if=is_valid_result, **kwargs)
bar_chart_per_number_of_original_states(results, 'timed_out', stat_method=np.sum, only_if=is_valid_result)
line_chart_per_number_of_original_states_per_alg_and_orac(results, 'timed_out', get_algorithm_names(results), get_oracle_names(results), stat_method=np.sum, only_if=is_valid_result)
bar_chart_per_algorithm_and_oracle(results, 'learning_rounds', stat_method=np.mean, 
                                   only_if=lambda res: is_valid_result(res) and res["timed_out"],
                                   title="Mean number of learning rounds for timed out calls", **kwargs)

def last_num_traces(result: dict):
    max_round = max(int(i) for i in result['detailed_learning_info'].keys())
    return result['detailed_learning_info'][str(max_round)]["num_traces"]

bar_chart_per_algorithm_and_oracle(results, last_num_traces, stat_method=np.mean, only_if=lambda res: is_valid_result(res) and res["timed_out"],
                                   title="Mean number of traces in last round for timed out calls", **kwargs)

bar_chart_per_algorithm_and_oracle(results, 'total_time', stat_method=np.sum, only_if=lambda res: is_valid_result(res) and res["timed_out"], **kwargs)

## Learning Rounds

In [ ]:
bar_chart_per_algorithm_and_oracle(results, "learning_rounds", stat_method=np.mean, only_if=is_valid_result, **kwargs)

## Runtime

### Overall

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

if GROUP_BY_MODEL:
    kwargs["group_by"] = "original_automaton"


bar_chart_per_algorithm_and_oracle(results, "total_time", stat_method=np.mean, only_if=is_valid_result, **kwargs)

### Per number of states

In [ ]:
ALGORITHMS = 'all'
ORACLES = ("None", )# 'all'
CHART_TYPE = 'line'  # one of 'bar', 'line'

algs = ALGORITHMS
if ALGORITHMS == 'all':
    algs = get_algorithm_names(results)

eq_oracles = ORACLES    
if ORACLES == 'all':
    eq_oracles = get_oracle_names(results)

glitch_percentages = get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result) 

for glitch_percentage in glitch_percentages:
    display(Markdown(f'#### {glitch_percentage}% glitches'))
    if CHART_TYPE == 'bar':
        for (a, o) in itertools.product(algs, eq_oracles):
            bar_chart_per_number_of_original_states(results, "total_time", stat_method=np.mean, 
                                                    only_if=lambda res: res['algorithm_name'] == a 
                                                                        and res['oracle'] == o
                                                                        and res['glitch_percent'] == glitch_percentage
                                                                        and is_valid_result(res))
            
    elif CHART_TYPE == 'line':
        line_chart_per_number_of_original_states_per_alg_and_orac(results, "total_time", stat_method=np.mean,
                                                                  algs=algs, oracles=eq_oracles,
                                                                  only_if=lambda res: is_valid_result(res)
                                                                                      and res['glitch_percent'] == glitch_percentage)
    else:
        raise NotImplementedError(f"Chart type {CHART_TYPE} not implemented")

## Heuristics

In [ ]:
def highest_score(learning_round_info: dict):
    if "heuristic_scores" not in learning_round_info:
        return -1
    return max(learning_round_info["heuristic_scores"].values())

def best_num_states(learning_round_info: dict):
    if "heuristic_scores" not in learning_round_info:
        return -1
    return int(max(learning_round_info["heuristic_scores"], key=learning_round_info["heuristic_scores"].get))

def distance_of_peak_to_true_num_states(result: dict, step: int):
    peak_num_states = best_num_states(result["detailed_learning_info"][str(step)])
    true_num_states = result["original_automaton_size"]
    if peak_num_states == -1:
        if step == 1:
            return result["original_automaton_num_outputs"] - true_num_states
        else:
            return distance_of_peak_to_true_num_states(result, step - 1)
    return peak_num_states - true_num_states

for alg, orac in itertools.product(get_algorithm_names(results), get_oracle_names(results)):
    display(Markdown(f'#### {alg}/{orac}'))
    line_chart_over_learning_rounds(results, key=distance_of_peak_to_true_num_states,
                                    only_if=lambda res: is_valid_result(res) 
                                                        and res["algorithm_name"] == alg 
                                                        and res["oracle"] == orac, 
                                    get_from_full_result=True, 
                                    colorcode_by_alg=False)
    #line_chart_over_learning_rounds(results, key=highest_score,
    #                                only_if=lambda res: is_valid_result(res) 
    #                                                    and res["algorithm_name"] == alg 
    #                                                    and res["oracle"] == orac, 
    #                                stay_at_last_val=lambda val: val == -1,
    #                                colorcode_by_alg=False)
    #line_chart_over_learning_rounds(results, key=best_num_states, 
    #                                only_if=lambda res: is_valid_result(res) 
    #                                                    and res["algorithm_name"] == alg 
    #                                                    and res["oracle"] == orac, 
    #                                stay_at_last_val=lambda val: val == -1,
    #                                colorcode_by_alg=False)

In [ ]:
kwargs = {}
if len(get_all_possible_values_for_key(results, "glitch_percent", only_if=is_valid_result)) > 1:
    kwargs["group_by"] = "glitch_percent" 

def num_rounds_with_returned_peak(result: dict):
    returned_peak = best_num_states(result["detailed_learning_info"][str(result["learning_rounds"])])
    c = 0
    for step in range(result["learning_rounds"], 0, -1):
        if best_num_states(result["detailed_learning_info"][str(step)]) == returned_peak:
            c += 1
        else:
            break
    return c
        
bar_chart_per_algorithm_and_oracle(results, num_rounds_with_returned_peak, stat_method=np.mean, only_if=is_valid_result, **kwargs)

## Transition / State Coverage

In [ ]:
from aalpy import load_automaton_from_file
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def analyze_algorithms(results):
    """Analyzes and compares transition behavior across multiple algorithms."""
    
    # Organize data by algorithm
    algorithm_data = defaultdict(list)
    original_automata = {}

    for entry in results:
        algo_name = entry["algorithm_name"]
        taken_transitions = entry["sul_transitions"]
        original_automaton = load_automaton_from_file(entry["original_automaton"], "moore")

        algorithm_data[algo_name].append(taken_transitions)
        original_automata[algo_name] = original_automaton  # Assume all runs of algo use same original automaton
    
    # Aggregate statistics
    transition_counts = {algo: defaultdict(list) for algo in algorithm_data}
    extra_transitions = {algo: defaultdict(int) for algo in algorithm_data}

    for algo, runs in algorithm_data.items():
        # original_transitions = original_automata[algo]["transitions"]  # Assuming this structure

        for run in runs:
            for state, inputs in run.items():
                for inp, transitions in inputs.items():
                    for target, count in transitions.items():
                        transition_counts[algo][(state, inp, target)].append(count)

                        # # Count extra transitions (not in original automaton)
                        # if (state, inp, target) not in original_transitions:
                        #     extra_transitions[algo][(state, inp, target)] += count

    # Compute summary statistics
    summary_stats = {}
    for algo, counts in transition_counts.items():
        all_counts = [np.sum(v) for v in counts.values()]
        summary_stats[algo] = {
            "min": np.min(all_counts),
            "max": np.max(all_counts),
            "mean": np.mean(all_counts),
            "median": np.median(all_counts),
            "std_dev": np.std(all_counts),
            # "extra_transitions": sum(extra_transitions[algo].values())
        }

    # Print statistics
    print("\n### Transition Statistics per Algorithm ###\n")
    for algo, stats in summary_stats.items():
        print(f"Algorithm: {algo}")
        print(f"  Min Transitions: {stats['min']}")
        print(f"  Max Transitions: {stats['max']}")
        print(f"  Mean Transitions: {stats['mean']:.2f}")
        print(f"  Median Transitions: {stats['median']}")
        print(f"  Std Dev: {stats['std_dev']:.2f}")
        # print(f"  Extra Transitions: {stats['extra_transitions']}")
        print("-" * 40)

    # Plot comparison
    plot_transition_distributions(transition_counts)
    plot_transition_differences(transition_counts)

def plot_transition_distributions(transition_counts):
    """Plots bar graphs of transition frequencies per algorithm."""
    plt.figure(figsize=(10, 5))
    
    for algo, counts in transition_counts.items():
        all_counts = [np.sum(v) for v in counts.values()]
        plt.hist(all_counts, bins=20, alpha=0.5, label=algo)

    plt.xlabel("Transition Count")
    plt.ylabel("Frequency")
    plt.title("Transition Frequency Distribution")
    plt.legend()
    plt.show()

def plot_transition_differences(transition_counts):
    """Plots scatter plots to compare transition differences between two algorithms."""
    if len(transition_counts) != 2:
        return  # Only makes sense for 2 algorithms

    algo1, algo2 = list(transition_counts.keys())
    shared_transitions = set(transition_counts[algo1].keys()) & set(transition_counts[algo2].keys())

    diffs = []
    for t in shared_transitions:
        mean_algo1 = np.mean(transition_counts[algo1][t])
        mean_algo2 = np.mean(transition_counts[algo2][t])
        diffs.append((mean_algo1, mean_algo2))

    x, y = zip(*diffs)

    plt.figure(figsize=(6, 6))
    plt.scatter(x, y, alpha=0.6)
    plt.plot([min(x), max(x)], [min(x), max(x)], 'r--')  # Identity line
    plt.xlabel(f"{algo1} Mean Transitions")
    plt.ylabel(f"{algo2} Mean Transitions")
    plt.title("Per-Transition Differences")
    plt.show()

# analyze_algorithms(results)  # ai generated - not incredibly useful currently

In [ ]:
from aalpy.utils import load_automaton_from_file
from aalpy import MooreMachine
import matplotlib.colors as mcolors


def transition_statistics(result: dict,
                          transition_coverage_frequency_threshold = 1,
                          transition_coverage_frequency_threshold_mode = 'absolute'):
    transitions = TracedMooreSUL.flatten_transitions_dict(result["sul_transitions"])

    original_automaton: MooreMachine = load_automaton_from_file(result["original_automaton"], "moore")

    glitched_freqs = []
    dominant_freqs = []
    for (s1, l, s2), trans_count in transitions.items():
        s1 = original_automaton.get_state_by_id(s1)
        s2 = original_automaton.get_state_by_id(s2)
        
        if l not in s1.transitions:
            # JSON serialization made our int-transitions into strings - revert
            l = int(l)
            
        if s1.transitions[l] != s2:
            glitched_freqs.append(trans_count)
        else:
            dominant_freqs.append(trans_count)

    total_num_glitches = sum(g for g in glitched_freqs)
    total_num_dominant = sum(d for d in dominant_freqs)
    total_num_steps = total_num_glitches + total_num_dominant
    # assert total_num_steps == result["steps_learning"], f"number of steps does not match" # TODO un-comment

    glitch_percentage = total_num_glitches / total_num_steps * 100

    possible_transitions = 0
    taken_transitions = 0

    if transition_coverage_frequency_threshold_mode == 'absolute':
        threshold = transition_coverage_frequency_threshold
    elif transition_coverage_frequency_threshold_mode == 'relative':
        threshold = total_num_steps * transition_coverage_frequency_threshold
    else:
        raise NotImplementedError(transition_coverage_frequency_threshold_mode)

    for s1 in original_automaton.states:
        for l, s2 in s1.transitions.items():
            possible_transitions += 1

            if (s1.state_id, l, s2.state_id) not in transitions:
                # see above; JSON serialization
                l = str(l)
            
            if transitions[(s1.state_id, l, s2.state_id)] >= threshold:
                taken_transitions += 1

    return {
        "glitched_freqs": glitched_freqs,
        "dominant_freqs": dominant_freqs,

        "glitch_percentage": glitch_percentage,

        "total_num_steps": total_num_steps,
        "total_num_glitches": total_num_glitches,
        "total_num_dominant": total_num_dominant,

        "max_glitched_freq": max(glitched_freqs),
        "mean_glitched_freq": np.mean(glitched_freqs),
        "median_glitched_freq": np.median(glitched_freqs),

        "min_dominant_freq": min(dominant_freqs),
        "max_dominant_freq": max(dominant_freqs),
        "mean_dominant_freq": np.mean(dominant_freqs),
        "median_dominant_freq": np.median(dominant_freqs),

        "transition_coverage": taken_transitions / possible_transitions,
    }

def transition_stat(key: str):
    def fun(res):
        return transition_statistics(res)[key]
    fun.__name__ = key
    return fun

def n_frequent_transition_coverage(threshold: int):
    def fun(res):
        return transition_statistics(res, threshold,
                                     transition_coverage_frequency_threshold_mode='absolute')["transition_coverage"]
    fun.__name__ = f"{threshold}-frequent transition coverage"
    return fun

def relative_transition_coverage(threshold: float):
    def fun(res):
        return transition_statistics(res, threshold/100,
                                     transition_coverage_frequency_threshold_mode='relative')["transition_coverage"]
    fun.__name__ = f"relative transition coverage (at least {threshold}% of steps)"
    return fun

# bar_chart_per_algorithm(results, transition_stat("transition_coverage"), stat_method=np.median)
# bar_chart_per_algorithm(results, n_frequent_transition_coverage(4), stat_method=np.mean)
# bar_chart_per_algorithm(results, relative_transition_coverage(2), stat_method=np.median)
# scatterplot_per_alg(results, x_key=transition_stat("min_dominant_freq"), y_key=transition_stat("max_glitched_freq"))

def plot_transition_coverage_over_threshold(results: list[dict], mode: str, start: float, end: float, step: float, stat_method: np.mean):
    algs = get_algorithm_names(results)
    eq_oracles = get_oracle_names(results)

    plt.figure(figsize=(10, 6))

    def add_labels(line):
        x, y = line.get_data()
        for x, y in zip(x, y):
            plt.annotate(f'{y:.2f}', (x, y))

    marker_per_alg = {alg: marker for alg, marker in
                      zip(algs, ['o', '*', 'p', 'h', 'd', 'v', '8', 's', 'P', 'H', 'X', '1', '2', '3' ])}
    color_per_alg =  {alg: color for alg, color in
                      zip(algs, mcolors.TABLEAU_COLORS)}
    line_style_per_oracle = {orac: style for orac, style in
                             zip(eq_oracles, ['-', ':', '-.', '--'])}

    ao_to_values = {(a, o): [] for (a, o) in itertools.product(algs, eq_oracles)}
    x_axis = np.arange(start, end, step)
    for (a, o) in ao_to_values.keys():
        values = []
        for threshold in x_axis:
            fun = n_frequent_transition_coverage(threshold) if mode == 'absolute' else relative_transition_coverage(threshold)
            values_at_threshold = [fun(res) for res in results
                                   if res['algorithm_name'] == a
                                   and res['oracle'] == o]
            values.append(stat_method(values_at_threshold))
        alg_str = f"{a}" if len(eq_oracles) == 1 else f"{a}, {o}"
        fmt = f"{line_style_per_oracle[o]}{marker_per_alg[a]}"
        color = color_per_alg[a]
        line, = plt.plot(x_axis, values, fmt, label=alg_str, color=color)
        add_labels(line)

    mode_str = "N-frequent transition coverage" if mode == 'absolute' else "Relative transition coverage (at least N %)"
    plt.xlabel(f"N (minimum {'number' if mode=='absolute' else 'percentage'} of steps for a transition to count as covered)")
    plt.ylabel('Transition coverage')
    plt.title(f"{mode_str} ({stat_method.__name__} over results)")
    plt.legend()
    plt.tight_layout()
    plt.show()


plot_transition_coverage_over_threshold(results, 'absolute', 1, 15, 1, stat_method=np.mean)
plot_transition_coverage_over_threshold(results, 'relative', 1, 5, 0.5, stat_method=np.mean)